In [4]:
import os
import random
import string
import re
import logging
import time
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

/Users/alejandronogales/anaconda3/envs/traductor/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-06 22:08:29.333733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r", encoding = 'utf8') as f:
        data = f.read()
    return data.split('\n')

kichwa_sentences = load_data('data/input/kichwa_vocab.txt')
spanish_sentences = load_data('data/input/spanish_vocab.txt')

In [6]:
pares_oraciones = []
for index, item in enumerate(kichwa_sentences, start=0):
    kichwa = item
    spanish = "[start] " + spanish_sentences[index] + " [end]"
    pares_oraciones.append((kichwa, spanish))

In [7]:
random.shuffle(pares_oraciones)
num_pares_validar = int(0.1 * len(pares_oraciones))
num_pares_train = int(0.9 * len(pares_oraciones))
pares_train = pares_oraciones[:num_pares_train]
pares_validar = pares_oraciones[num_pares_train:num_pares_train + num_pares_validar]

In [8]:
modificar_chars = string.punctuation + "¿"
modificar_chars = modificar_chars.replace("[", "")
modificar_chars = modificar_chars.replace("]", "")

len_sentences = 7
num_vocab = 5000

def get_standardization(text):
    texto = tf.strings.lower(text)
    return tf.strings.regex_replace(
        texto, f"[{re.escape(modificar_chars)}]", "")

texto_source_vectorizado = layers.TextVectorization(
    output_mode = "int",
    output_sequence_length = len_sentences,
    max_tokens = num_vocab
)
texto_target_vectorizado = layers.TextVectorization(
    output_mode = "int",
    output_sequence_length = len_sentences + 1,
    max_tokens = num_vocab,
    standardize = get_standardization
)
textos_train_kichwa = [par[0] for par in pares_train]
textos_train_spanish = [par[1] for par in pares_train]
texto_source_vectorizado.adapt(textos_train_kichwa)
texto_target_vectorizado.adapt(textos_train_spanish)

In [9]:
batch_size = 64

def set_format(kichwa, spanish):
    kichwa = texto_source_vectorizado(kichwa) 
    spanish = texto_target_vectorizado(spanish)
    return (
        {
            "kichwa": kichwa,
            "spanish": spanish[:, :-1],
        }, 
        spanish[:, 1:]
    )

def make_dataset(pairs):
    textos_kichwa, textos_spanish = zip(*pairs)
    textos_kichwa = list(textos_kichwa)
    textos_spanish = list(textos_spanish)

    dataset = tf.data.Dataset.from_tensor_slices((textos_kichwa, textos_spanish))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(set_format, num_parallel_calls=4)

    return dataset.shuffle(2048).prefetch(16).cache()

dataset_train = make_dataset(pares_train)
dataset_validacion = make_dataset(pares_validar)

for entradas, salidas in dataset_train.take(1):
    print(f"entradas['kichwa'].shape: {entradas['kichwa'].shape}")
    print(f"entradas['spanish'].shape: {entradas['spanish'].shape}")
    print(f"salidas.shape: {salidas.shape}")

entradas['kichwa'].shape: (64, 7)
entradas['spanish'].shape: (64, 7)
salidas.shape: (64, 7)


2023-08-06 22:08:56.149910: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [10]:
class EncoderTrans(layers.Layer):

    def __init__(self, num_heads, dense_dim, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.embed_dim = embed_dim
        self.attention = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(
                dense_dim, 
                activation = "relu"
            ),
            layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, entradas, mask = None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_salida = self.attention(entradas, entradas, attention_mask = mask)
        proj_entrada = self.layernorm_1(entradas + attention_salida)
        proj_salida = self.dense_proj(proj_entrada)

        return self.layernorm_2(proj_entrada + proj_salida)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "embed_dim": self.embed_dim
        })
        return config

In [11]:
class DecoderTrans(layers.Layer):
    def __init__(self, num_heads, dense_dim, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.embed_dim = embed_dim
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation = "relu"),
             layers.Dense(embed_dim)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "embed_dim": self.embed_dim
        })
        return config

    def get_mask_attention_causal(self, entradas):
        shape_entrada = tf.shape(entradas)
        batch_size, len_sequence = shape_entrada[0], shape_entrada[1]
        i = tf.range(len_sequence)[:, tf.newaxis]
        j = tf.range(len_sequence)
        mask = tf.cast(i >= j, dtype = "int32")
        mask = tf.reshape(mask, (1, shape_entrada[1], shape_entrada[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype = tf.int32)], axis = 0)
        return tf.tile(mask, mult)

    def call(self, entradas, salidas_encoder, mask = None):
        mask_causal = self.get_mask_attention_causal(entradas)
        if mask is not None:
            mask_padding = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            mask_padding = tf.minimum(mask_padding, mask_causal)
        attention_salida_1 = self.attention_1(
            query = entradas,
            value = entradas,
            key = entradas,
            attention_mask = mask_causal)
        attention_salida_1 = self.layernorm_1(entradas + attention_salida_1)
        attention_salida_2 = self.attention_2(
            query = attention_salida_1,
            value = salidas_encoder,
            key = salidas_encoder,
            attention_mask = mask_padding,
        )
        attention_salida_2 = self.layernorm_2(
            attention_salida_1 + attention_salida_2)
        proj_salida = self.dense_proj(attention_salida_2)

        return self.layernorm_3(attention_salida_2 + proj_salida)

In [12]:
class PositionalEncoding(layers.Layer):

    def __init__(self, dim_entrada, dim_salida, len_sentences, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = dim_entrada
        self.output_dim = dim_salida
        self.token_embeddings = layers.Embedding(
            input_dim = dim_entrada, output_dim = dim_salida)
        self.position_embeddings = layers.Embedding(
            input_dim = len_sentences, output_dim = dim_salida)
        self.sequence_length = len_sentences

    def call(self, entradas):
        limite = tf.shape(entradas)[-1]
        posiciones = tf.range(start = 0, limit = limite, delta = 1)
        tokens_embedded = self.token_embeddings(entradas)
        posiciones_embedded = self.position_embeddings(posiciones)
        return tokens_embedded + posiciones_embedded

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length
        })
        return config

In [13]:
num_heads = 8
embed_dim = 256
dense_dim = 2048

entradas_encoder = keras.Input(shape=(None,), dtype="int64", name="kichwa")
x = PositionalEncoding(num_vocab, embed_dim, len_sentences)(entradas_encoder)
salidas_encoder = EncoderTrans(num_heads, dense_dim, embed_dim )(x)

entradas_decoder = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEncoding(num_vocab, embed_dim, len_sentences,)(entradas_decoder)
x = DecoderTrans(num_heads, dense_dim, embed_dim )(x, salidas_encoder)
x = layers.Dropout(0.5)(x)

salidas_decoder = layers.Dense(num_vocab, activation="softmax")(x)
transformer = keras.Model([entradas_encoder, entradas_decoder], salidas_decoder)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

historial = transformer.fit(dataset_train, epochs = 200, validation_data = dataset_validacion)

Epoch 1/200
52/52 [==============================] - 25s 418ms/step - loss: 5.1582 - accuracy: 0.3435 - val_loss: 4.3683 - val_accuracy: 0.3916
Epoch 2/200
52/52 [==============================] - 23s 437ms/step - loss: 4.3435 - accuracy: 0.3896 - val_loss: 4.1621 - val_accuracy: 0.4066
Epoch 3/200
52/52 [==============================] - 23s 440ms/step - loss: 3.9238 - accuracy: 0.4265 - val_loss: 3.8488 - val_accuracy: 0.4501
Epoch 4/200
52/52 [==============================] - 24s 469ms/step - loss: 3.5545 - accuracy: 0.4650 - val_loss: 4.0933 - val_accuracy: 0.3999
Epoch 5/200
52/52 [==============================] - 24s 461ms/step - loss: 3.1971 - accuracy: 0.5056 - val_loss: 3.6155 - val_accuracy: 0.4754
Epoch 6/200
52/52 [==============================] - 22s 426ms/step - loss: 2.8427 - accuracy: 0.5498 - val_loss: 3.4748 - val_accuracy: 0.5085
Epoch 7/200
52/52 [==============================] - 23s 440ms/step - loss: 2.5477 - accuracy: 0.5930 - val_loss: 3.4087 - val_accuracy:

52/52 [==============================] - 24s 459ms/step - loss: 0.0431 - accuracy: 0.9905 - val_loss: 4.2235 - val_accuracy: 0.5473
Epoch 58/200
52/52 [==============================] - 24s 467ms/step - loss: 0.0425 - accuracy: 0.9901 - val_loss: 4.1387 - val_accuracy: 0.5329
Epoch 59/200
52/52 [==============================] - 25s 481ms/step - loss: 0.0474 - accuracy: 0.9896 - val_loss: 4.3256 - val_accuracy: 0.5463
Epoch 60/200
52/52 [==============================] - 23s 449ms/step - loss: 0.0609 - accuracy: 0.9877 - val_loss: 4.2051 - val_accuracy: 0.5385
Epoch 61/200
52/52 [==============================] - 21s 400ms/step - loss: 0.0599 - accuracy: 0.9869 - val_loss: 4.1656 - val_accuracy: 0.5463
Epoch 62/200
52/52 [==============================] - 21s 402ms/step - loss: 0.0500 - accuracy: 0.9886 - val_loss: 4.2061 - val_accuracy: 0.5427
Epoch 63/200
52/52 [==============================] - 21s 408ms/step - loss: 0.0454 - accuracy: 0.9901 - val_loss: 4.3099 - val_accuracy: 0.533

52/52 [==============================] - 36s 700ms/step - loss: 0.0297 - accuracy: 0.9924 - val_loss: 4.4967 - val_accuracy: 0.5551
Epoch 114/200
52/52 [==============================] - 37s 703ms/step - loss: 0.0345 - accuracy: 0.9917 - val_loss: 4.4437 - val_accuracy: 0.5484
Epoch 115/200
52/52 [==============================] - 36s 702ms/step - loss: 0.0301 - accuracy: 0.9929 - val_loss: 4.4446 - val_accuracy: 0.5499
Epoch 116/200
52/52 [==============================] - 36s 694ms/step - loss: 0.0337 - accuracy: 0.9914 - val_loss: 4.4625 - val_accuracy: 0.5494
Epoch 117/200
52/52 [==============================] - 36s 697ms/step - loss: 0.0307 - accuracy: 0.9921 - val_loss: 4.4496 - val_accuracy: 0.5447
Epoch 118/200
52/52 [==============================] - 36s 701ms/step - loss: 0.0357 - accuracy: 0.9915 - val_loss: 4.4382 - val_accuracy: 0.5463
Epoch 119/200
52/52 [==============================] - 37s 704ms/step - loss: 0.0319 - accuracy: 0.9922 - val_loss: 4.5360 - val_accuracy:

52/52 [==============================] - 36s 684ms/step - loss: 0.0304 - accuracy: 0.9926 - val_loss: 4.6391 - val_accuracy: 0.5401
Epoch 170/200
52/52 [==============================] - 36s 691ms/step - loss: 0.0258 - accuracy: 0.9940 - val_loss: 4.6231 - val_accuracy: 0.5489
Epoch 171/200
52/52 [==============================] - 35s 678ms/step - loss: 0.0279 - accuracy: 0.9929 - val_loss: 4.6653 - val_accuracy: 0.5504
Epoch 172/200
52/52 [==============================] - 35s 679ms/step - loss: 0.0258 - accuracy: 0.9932 - val_loss: 4.6494 - val_accuracy: 0.5479
Epoch 173/200
52/52 [==============================] - 36s 686ms/step - loss: 0.0281 - accuracy: 0.9922 - val_loss: 4.7191 - val_accuracy: 0.5422
Epoch 174/200
52/52 [==============================] - 36s 690ms/step - loss: 0.0270 - accuracy: 0.9931 - val_loss: 4.6115 - val_accuracy: 0.5453
Epoch 175/200
52/52 [==============================] - 36s 690ms/step - loss: 0.0273 - accuracy: 0.9929 - val_loss: 4.6317 - val_accuracy:

In [ ]:
# save the model
transformer.save('data/output/model_kichwa_spanish')

# save the pares train to use on the server
with open('data/output/pares_kichwa_spanish.txt', 'w') as f:
    for par in pares_train:
        par_string = ''.join(par)
        f.write(par_string)
        f.write('\n')

print("Saved model to disk")

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('epoch')
plt.ylabel('perdida')
plt.plot(historial.history['loss'])

In [ ]:
vocab_spanish = texto_target_vectorizado.get_vocabulary()
busqueda_spanish = dict(zip(range(len(vocab_spanish)), vocab_spanish))
max_len_sentence = 7


def traducir(input_sentence):
    tokenized_input_sentence = texto_source_vectorizado([input_sentence])
    sentence_decoded = "[start]"
    for i in range(max_len_sentence):
        sentence_target_tokenized = texto_target_vectorizado(
            [sentence_decoded])[:, :-1]
        predicciones = transformer(
            [tokenized_input_sentence, sentence_target_tokenized])
        index_token = np.argmax(predicciones[0, i, :])
        token = busqueda_spanish[index_token]
        sentence_decoded += " " + token
        if token == "[end]":
            break
    return sentence_decoded

test_kichwa_texts = [pair[0] for pair in pares_train]

for _ in range(10):
    input_sentence = random.choice(test_kichwa_texts)
    print(input_sentence)
    print(traducir(input_sentence))
    print("...")

In [ ]:
print(traducir("hatun huasi"))